# DOCUMENTOS con Gemini

## Importaciones

In [1]:
import os
from dotenv import load_dotenv
import google.generativeai as genai
import io
import pathlib, base64, httpx

## Clave de la API

In [2]:
# Cargamos las variables de entorno desde .env e inicialiÇzamos la constante API_KEY
load_dotenv(override=True)
API_KEY = os.getenv('GEMINI_API_KEY') #si se llama así, no hace falta introducirla en el constructor de OpenAI

if not API_KEY:
    print("ERROR: No se ha encontrado la clave de la API")
else:
    print("Se ha encontrado la clave de la API correctamente")

Se ha encontrado la clave de la API correctamente


## Iniciación

In [3]:
MODEL_NAME = "gemini-1.5-flash"
genai.configure(api_key=API_KEY) 

In [4]:
system_prompt = "Utiliza un tono amable"
#system_prompt = "Utiliza un tono enfadado"

In [5]:
model = genai.GenerativeModel(
    model_name = MODEL_NAME,
    system_instruction = system_prompt,
    #generation_config = genai.GenerationConfig(
    #    max_output_tokens = 1000,
    #    temperature = 0.1,
    #),
)

## Un documento en local

In [6]:
doc = pathlib.Path('recursos/paper.pdf').read_bytes()

In [7]:
user_prompt = "Por favor, explícame que hay en el documento. Trata de detallar tanto como puedas. Además ponle un título"

In [8]:
response = model.generate_content([{'mime_type':'application/pdf', 'data': doc}, user_prompt])

In [9]:
print(response.text)

Claro que sí, aquí tienes una descripción detallada del documento proporcionado.

**Título:** Elaboración de un Artículo Científico de Investigación: Una Guía Paso a Paso

**Descripción del Documento:**

Este documento es un artículo científico en español que proporciona una guía detallada para escribir un artículo científico que reporte una investigación. Está dirigido a quienes se inician en la escritura científica y se divide en las siguientes secciones:

* **Resumen y Abstract:** Resumen en español e inglés de los objetivos y contenido del artículo.

* **Introducción:**  Explica el objetivo del artículo, que es guiar paso a paso en la escritura de un artículo científico, destacando los aspectos más relevantes.  Se discute la importancia de comunicar los resultados de la investigación para integrar conocimientos a la práctica profesional. También se abordan algunos mitos sobre publicaciones científicas y se indica la dificultad de la etapa final de la investigación (comunicación de 

## Un PDF en remoto

In [10]:
doc_path = "https://www.turismoasturias.es/documents/39908/3871406/Casonas%20Asturianas.pdf"
doc = base64.b64encode(httpx.get(doc_path).content).decode("utf-8")

In [11]:
user_prompt = "Por favor, resume el documento"

In [12]:
response = model.generate_content([{'mime_type':'application/pdf', 'data': doc}, user_prompt])

In [13]:
print(response.text)

Claro, aquí hay un resumen del documento que proporcionaste:

Este es un folleto para el Club de Calidad Casonas Asturianas, que presenta una colección de hoteles rurales de Asturias, España. El folleto está en español e inglés. Proporciona descripciones e imágenes de cada hotel, junto con información de contacto. El folleto también incluye un mapa que muestra la ubicación de cada hotel en Asturias.

Espero que esto ayude. Avísame si necesitas algo más.


## Un PDF en la API de File

In [14]:
doc_path = "https://www.turismoasturias.es/documents/39908/3871406/Casonas%20Asturianas.pdf"
doc = io.BytesIO(httpx.get(doc_path).content) #la API espera el archivo como entrada e BytesIO da una representación del archivo en memoria en lugar de en disco
doc_file = genai.upload_file(path=doc, mime_type='application/pdf')

In [15]:
user_prompt = "Por favor, resume el documento"

In [16]:
response = model.generate_content(
    [doc_file, user_prompt], #pueden ponerse varios documentos
)

In [17]:
print(response.text)

Claro, aquí tienes un resumen del folleto:

Este folleto presenta el Club de Calidad Casonas Asturianas, una colección de hoteles rurales en Asturias, España. El folleto destaca la historia y la calidad de los hoteles, y describe las comodidades y las actividades que ofrecen. Además, proporciona una breve descripción de las diferentes regiones de Asturias y los hoteles que se encuentran en cada una de ellas. También incluye un mapa que muestra la ubicación de los hoteles.

El folleto se dirige a los turistas que buscan una experiencia de lujo en un entorno rural. Es una herramienta de marketing que promociona el Club de Calidad Casonas Asturianas y sus hoteles miembros.


## Ver archivos subidos a la plataforma

In [18]:
for f in genai.list_files():
    print("  ", f)

   genai.File({
    'name': 'files/lace3jx5gwtt',
    'display_name': '',
    'mime_type': 'application/pdf',
    'sha256_hash': 'Y2JmZTAxZTIyZWEzYmFhNTAzOGJkZjAyNWQzZDNjNzRiYzVhZmY2MTA0OGRlZTcwZmViMmUwZTgwMzJmMWE1Mg==',
    'size_bytes': '6596651',
    'state': 'ACTIVE',
    'uri': 'https://generativelanguage.googleapis.com/v1beta/files/lace3jx5gwtt',
    'create_time': '2025-02-02T16:51:01.985315Z',
    'expiration_time': '2025-02-04T16:51:01.923845342Z',
    'update_time': '2025-02-02T16:51:01.985315Z'})
   genai.File({
    'name': 'files/rl30xamybg2k',
    'display_name': '',
    'mime_type': 'application/pdf',
    'sha256_hash': 'Y2JmZTAxZTIyZWEzYmFhNTAzOGJkZjAyNWQzZDNjNzRiYzVhZmY2MTA0OGRlZTcwZmViMmUwZTgwMzJmMWE1Mg==',
    'size_bytes': '6596651',
    'state': 'ACTIVE',
    'uri': 'https://generativelanguage.googleapis.com/v1beta/files/rl30xamybg2k',
    'create_time': '2025-02-02T16:48:49.968209Z',
    'expiration_time': '2025-02-04T16:48:49.913504251Z',
    'update_time': '2025-

## Borrar archivos subidos

In [19]:
for f in genai.list_files():
    f.delete()

## Ver archivos subidos

In [20]:
for f in genai.list_files():
    print("  ", f.name)